In [1]:
from tqdm import tqdm
from datasets import load_dataset, DatasetDict, concatenate_datasets
from IPython.display import HTML
from markdownify import markdownify as md
import re
import numpy as np

In [3]:
dataset = load_dataset("HuggingFaceH4/stack-exchange-preferences", num_proc=32)

KeyboardInterrupt: 

In [ ]:
HTML(ds[0]["question"])

In [ ]:
def lang_callback(el):
    lang = el['class'][0] if el.has_attr('class') else None
    
    if not lang is None:
        lang = lang.split("-")[-1]
    return lang

In [ ]:
def html2md(text):
    text = md(text, code_language_callback=lang_callback)
    text = re.sub(r"\n\s*\n", "\n\n", text).strip()
    return text.encode('utf-8', 'replace').decode()

In [ ]:
for i in range(10):
    text = html2md(ds[i]["question"])
    print(text)
    print("=="*10)

In [ ]:
np.mean([len(ds[i]["answers"])*(len(ds[i]["answers"]))/2 for i in range(2204)])

In [ ]:
ds[0]

In [ ]:
ds = ds.shuffle(seed=42)
index = list(range(len(ds)))

ds_splits = DatasetDict({
    "finetune": ds.select(index[:3_000_000]),
    "reward": ds.select(index[3_000_000:6_000_000]),
    "rl": ds.select(index[6_000_000:9_000_000]),
    "evaluation": ds.select(index[9_000_000:]),
})

In [ ]:
ds_splits

In [ ]:
def binary_comparison(answers):
    """Returns tuples of answers, first always best"""
    pairs = []
    
    for i in range(len(answers)-1):
        for j in range(i+1, len(answers)):
            if answers[i]["pm_score"]>answers[j]["pm_score"]:
                pairs.append((answers[i]["text"], answers[j]["text"]))
            elif answers[i]["pm_score"]<answers[j]["pm_score"]:
                pairs.append((answers[j]["text"], answers[i]["text"]))
    return pairs

In [ ]:
def preprocess(examples):
    """Cleans HTML and returns paired answers (j is better than k). Note that this returns more examples (one for each pair per question)."""
    
    MAX_PAIRS_PER_QUESTION = 10
    n_samples = len(examples["qid"])
    
    # initialize empty lists for new samples
    new_examples = {"question": [], "response_j": [], "response_k": []}
    for key in examples:
        new_examples[key] = []
    
    for sample_id in range(n_samples):
        # get pairs where first is always the better one
        pairs = binary_comparison(examples["answers"][sample_id])
        n_answers = len(examples["answers"][sample_id])
        
        # sample if we get more pairs than maximum
        if len(pairs) > MAX_PAIRS_PER_QUESTION:
            indices = np.random.choice(list(range(len(pairs))), MAX_PAIRS_PER_QUESTION, replace=False)
            pairs = [pairs[i] for i in indices]
        
        # construct the samples
        for pair in pairs:
            for key in examples:
                if key=="question":
                    new_examples[key].append(html2md(examples[key][sample_id]))
                else:
                    new_examples[key].append(examples[key][sample_id])
            new_examples["response_j"].append(html2md(pair[0]))
            new_examples["response_k"].append(html2md(pair[1]))
    return new_examples

In [ ]:
ds_result = ds_splits.map(preprocess, batch_size=1000, batched=True, num_proc=4)

In [ ]:
ds_result

In [ ]:
ds_result["finetune"][0]

In [ ]:
ds_result = ds_result.remove_columns(["answers"])

In [ ]:
ds_result['finetune'][0]

In [ ]:
for key in ds_result:
    print(key)

In [ ]:
import os
import time
from multiprocessing import Pool
from tqdm import tqdm

from huggingface_hub import Repository


def save_shard(shard_tuple):
    """Save shard"""
    filename, shard = shard_tuple
    # use to_json instead to save as json file
    shard.to_parquet(filename)


def save_manual_shards(ds, user="lvwerra", remote_dataset_repo="stack-exchange-paired", subfolder="train"):
    """Save sharded data
    Args:
        ds (Dataset): dataset to be saved
        user (str): user name
        remote_dataset_repo (str): remote dataset repository
        out_path (str): path to save the shards"""
    # this will create a folder OUT_PATH that is a clone of REMOTE_DATASET_REPO
    # you can save the shards inside it and do git add/commit/push to push data to the hub
    out_path = remote_dataset_repo
    # if out path doesnt already exist
    if not os.path.exists(out_path):
        repo = Repository(
            local_dir=out_path,
            clone_from=user + "/" + remote_dataset_repo,
            repo_type="dataset",
            private=False,
            use_auth_token=True,
            git_user=user,
        )

    # files will be numerous we save them in a folder called data inside out_path
    if not os.path.exists(out_path):
        os.mkdir(out_path + "/data")
    os.mkdir(out_path + f"/data/{subfolder}")
    
    SHARD_SIZE = 1000 << 20
    if ds._indices is not None:
        dataset_nbytes = ds.data.nbytes * len(ds._indices) / len(ds.data)
    else:
        dataset_nbytes = ds.data.nbytes
    num_shards = int(dataset_nbytes / SHARD_SIZE) + 1
    print(f"Number of shards: {num_shards}")

    print("sharding the dataset")
    t_start = time.time()
    shards = (
        ds.shard(num_shards=num_shards, index=i, contiguous=True)
        for i in range(num_shards)
    )
    # use f"{OUT_PATH}/data/train-{index:05d}-of-{num_shards:05d}.json" instead for json files
    filenames = (
        f"{out_path}/data/{subfolder}/train-{index:05d}-of-{num_shards:05d}.parquet"
        for index in range(num_shards)
    )

    with Pool(16) as p:
        list(
            tqdm(
                p.imap_unordered(save_shard, zip(filenames, shards), chunksize=4),
                total=num_shards,
            )
        )
    print(f"Time to save dataset: {time.time()-t_start:.2f}")
    # to push dataset to hub do: git add/commit/push inside OUT_PATH

In [ ]:
for key in ds_result:
    save_manual_shards(ds_result[key], subfolder=key)